<center>
<img src="logo.png" height="900"> 
</center>


# Байесовская домашка

В этом задании мы оценим несколько простых моделей с помощью `pymc3`. 

Задачи делались [на основе книги Байесовские методы для хакеров.](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers) В ней вы можете найти ещё больше примеров использования байесовских методов с помощью pymc3. 

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats as sts

import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import pymc3 as pm
pm.__version__

# Упражнение 1 (Маша и медведи)

Помните задачку с вынюхиванием из лекции? Давайте начнём с того, что попробуем решить её в `pymc3`.

__Модель медведя:__ $y_i \sim N(m, 2^2)$

__Априорное мнение, где спряталась Маша:__ $m \sim N(1, 4^2)$

In [ ]:
y = np.array([0.5, -1]) # медведи

Задайте априорное распределение и нашу модель.

In [ ]:
basic_model = pm.Model()

with basic_model:
    # your code here
    

Найдите моду апостериорного распределения (байесовские методы для бедных). Запишите результат в переменную `map_estimate`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

map_estimate = ...

# your code here


In [ ]:
assert np.abs(float(map_estimate['m']) + 0.111111) < 1e-4

# несколько похожих скрытых тестов

Обучите модельс с помощью MCMC, 500 итераций для обучения будет достаточно. 

In [ ]:
with basic_model:
    trace = ...
    # your code here
    

Посмотрите на визуализацию Марковских цепей. Правда ли, что цепь сошлась?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Найдите вероятность $P(m < 0)$. Ответ запишите в переменную `ans_prob`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


In [ ]:
assert ans_prob > 0.5

# несколько похожих скрытых тестов

> __Важно:__ перед тем, как отправить тетрадку на проверку, закоментируйте все части кода, где делаются долгие вычисления. На грейдер наложено ограничение в работе в 30 секунд и он может не успеть отработать. Из-за этого задание специфицировано таким образом, что вам нужно получить ответы на вопросы и вбить их в переменные.

# Упражнение 2 (Челленджер)

Байесовский подход - это просто другой способ оценивать те же самые модели. Например, давайте попробуем оценить логистическую регрессию. 

$28$ января $1986$ года космический шатл «Челленджер» взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для $23$ предшествовавших катастрофе полётов «Челленджера» известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

In [ ]:
df = pd.read_csv('challenger.csv', sep=',')
print(df.shape)
df.head()

In [ ]:
t = df.Temperature.values
y = df.Incident.values

plt.figure(figsize=(10, 5))
plt.scatter(t, y, s=75, alpha=0.5)
plt.yticks([0, 1])

plt.ylabel("Была ли поломка")
plt.xlabel("Температура колец");


Обучим Байесовскую версию логистичесской регресcии.  Модель буде выглядеть следующим образом: 

\begin{equation*}
\begin{aligned}
& \beta \sim N(0, 10^2) \\
& \alpha \sim N(0, 10^2) \\
& p \sim Bern(p) \\
& p = P(y = 1) = \frac{1}{1 + e^{\alpha + \beta \cdot t}}
\end{aligned}
\end{equation*}

In [ ]:
import theano.tensor as tt  # под капотом пакета работает theano 
                            # именно на нём прописываются разные сложные формулы

t = df.Temperature.values
y = df.Incident.values

with pm.Model() as model:
    
    beta = ...
    alpha = ...
    # your code here
    
    p = pm.Deterministic("p", 1.0/(1. + tt.exp(beta*t + alpha)))
    y_obs = ...
    # your code here
    

Найдите моду апостериорного распределения (байесовские методы для бедных). Запишите результат в переменную `map_estimate`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

map_estimate = ...

# your code here


In [ ]:
assert np.abs(float(map_estimate['beta']) - 0.1665) < 1e-2

# несколько похожих скрытых тестов

Попробуем обучить модель немного по-новому. В переменной `step` запишемм конкретный алгоритм, который будем использовать при обучении. В `pm.sapmple` укажем из какой точке надо стартовать алгоритму. В качестве стартовой точки укажем моду апостериорного распределения, полученную в предыдущем пункте. 

In [ ]:
with model:
    step = pm.Metropolis()
    trace = pm.sample(120000, step=step, start=map_estimate)
    burned_trace = trace[100000::10]  # оставляем каждое 10 наблюдение, чтобы соседние наблюдения были менее зависимы

Посмотрим, сошлись ли цепи.

In [ ]:
az.plot_trace(trace);

Какова вероятность того, что $\beta < 0.0001$? То есть, какова вероятность того, что между температурой и вероятностью поломки нет связи? Запишите овтет в переменную `prob_beta`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

prob_beta = ...

# your code here


In [ ]:
assert prob_beta < 0.1

# несколько похожих скрытых тестов

Нарисуем Получившиеся результаты. Для каждой температуры найдем $95\%$ байесовский интервал для вероятности того, что катастрофа произойдёт.

In [ ]:
from scipy.stats.mstats import mquantiles

def logistic(x, beta, alpha=0):
    return 1.0 / (1.0 + np.exp(np.dot(beta, x) + alpha))

t_seq = np.linspace(t.min() - 5, t.max()+5, 50)[:, None]
p_seq = logistic(t_seq.T, burned_trace['beta'][:,None], burned_trace['alpha'][:,None])

mean_prob_t = p_seq.mean(axis=0)
qs = mquantiles(p_seq, [0.025, 0.975], axis=0)

plt.figure(figsize=(10,5))
plt.fill_between(t_seq[:, 0], *qs, alpha=0.7, color="#7A68A6")
plt.plot(t_seq[:, 0], qs[0], label="95% интервал", color="#7A68A6", alpha=0.7)
plt.plot(t_seq, mean_prob_t, lw=1, ls="--", color="k", label="среднее")
plt.legend(loc="lower left")
plt.scatter(t, y, color="k", s=50, alpha=0.5)

plt.xlabel("Температура, $t$")
plt.ylabel("Вероятность поломки");

В день катастрофы температура уплотнителя была $31$ градус по Фарингейту. Найдите апостерионое распределение  вероятности катастрофы. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Найдите $5\%$ квантиль, математическое ожидание и $95\%$ квантиль апостериорного распределения. Заишите их в соответствующие переменные. Что это означает в контексте прогноза катастрофы? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

q05 = ...
avg = ...
q95 = ...

# your code here


In [ ]:
assert q05  > 0.8

# несколько похожих скрытых тестов

> __Важно:__ перед тем, как отправить тетрадку на проверку, закоментируйте все части кода, где делаются долгие вычисления. На грейдер наложено ограничение в работе в 30 секунд и он может не успеть отработать. Из-за этого задание специфицировано таким образом, что вам нужно получить ответы на вопросы и вбить их в переменные.